In [1]:
from lib.data import *

In [3]:
G_list = load_G_list(data_path='data/rome', index_file='data_index.txt', cache='G_list.pickle')

KeyboardInterrupt: 

In [ ]:
data_list = generate_data_list(G_list, 
                               sparse=True, 
                               pivot_mode='mis',
                               init_mode='random',
                               edge_index='full_edge_index',
                               edge_attr='full_edge_attr',
                               pmds_list=pickle.load(open('pmds_list.pickle', 'rb')),
                               device='cpu')